In [2]:
from os.path import join
import pandas as pd
import numpy as np

In [3]:
queries_dir = join("data", "queries")

In [4]:
cc_df = pd.read_csv(join(queries_dir, "q02_get_field_year_percentiles.csv"), index_col=0)

In [5]:
cc_df["percent_including_row"] = np.nan
cc_df["percent_excluding_row"] = np.nan
cc_df["cp_in"] = np.nan
cc_df["cp_ex"] = np.nan

In [6]:
for (source, field, year), group_df in cc_df.groupby(["source", "field", "year"]):
    total_num_papers = group_df["paper_count"].sum()
    cum_pct_in = 0.0
    cum_pct_ex = 0.0
    pct_excl_row = 0.0
    for group_row_i, row in group_df.sort_values(by="citation_count").reset_index().iterrows():
        global_row_i = row["index"]
        pct_incl_row = row["paper_count"] / total_num_papers * 100
        cum_pct_in += pct_incl_row
        
        cc_df.at[global_row_i, "percent_including_row"] = pct_incl_row
        cc_df.at[global_row_i, "percent_excluding_row"] = pct_excl_row
        cc_df.at[global_row_i, "cp_in"] = cum_pct_in
        cc_df.at[global_row_i, "cp_ex"] = cum_pct_ex
       
        cum_pct_ex += pct_incl_row
        pct_excl_row = pct_incl_row

In [7]:
cc_df.to_csv(join(queries_dir, "cumulative_frequencies.csv"))

In [9]:
paper_counts_df = cc_df.groupby(["source", "field", "year"]).sum()[["paper_count"]].reset_index()

In [10]:
paper_counts_df.to_csv(join(queries_dir, "paper_counts.csv"))

In [18]:
all_year_df = cc_df.loc[cc_df["year"] <= 2022]
all_year_sum_df = all_year_df.groupby(["source", "field", "citation_count"]).sum()[["paper_count"]].reset_index()
all_year_sum_df.head()

,source,field,citation_count,paper_count
0,external,Art,0,527921
1,external,Art,1,67370
2,external,Art,2,26129
3,external,Art,3,13424
4,external,Art,4,7954


In [19]:
for (source, field), group_df in all_year_sum_df.groupby(["source", "field"]):
    total_num_papers = group_df["paper_count"].sum()
    cum_pct_in = 0.0
    cum_pct_ex = 0.0
    pct_excl_row = 0.0
    for group_row_i, row in group_df.sort_values(by="citation_count").reset_index().iterrows():
        global_row_i = row["index"]
        pct_incl_row = row["paper_count"] / total_num_papers * 100
        cum_pct_in += pct_incl_row
        
        all_year_sum_df.at[global_row_i, "percent_including_row"] = pct_incl_row
        all_year_sum_df.at[global_row_i, "percent_excluding_row"] = pct_excl_row
        all_year_sum_df.at[global_row_i, "cp_in"] = cum_pct_in
        all_year_sum_df.at[global_row_i, "cp_ex"] = cum_pct_ex
       
        cum_pct_ex += pct_incl_row
        pct_excl_row = pct_incl_row

In [20]:
all_year_sum_df

,source,field,citation_count,paper_count,percent_including_row,percent_excluding_row,cp_in,cp_ex
0,external,Art,0,527921,78.666669,0.000000,78.666669,0.000000
1,external,Art,1,67370,10.038952,78.666669,88.705620,78.666669
2,external,Art,2,26129,3.893540,10.038952,92.599160,88.705620
3,external,Art,3,13424,2.000340,3.893540,94.599500,92.599160
4,external,Art,4,7954,1.185243,2.000340,95.784743,94.599500
...,...,...,...,...,...,...,...,...
50078,s2-fos-model,Sociology,8664,1,0.000159,0.000159,99.999364,99.999205
50079,s2-fos-model,Sociology,8884,1,0.000159,0.000159,99.999523,99.999364
50080,s2-fos-model,Sociology,11339,1,0.000159,0.000159,99.999682,99.999523
50081,s2-fos-model,Sociology,13131,1,0.000159,0.000159,99.999841,99.999682


In [21]:
all_year_sum_df.to_csv(join(queries_dir, "cumulative_frequencies_all_year_sum.csv"))